In [ ]:
%pip install -r requirements.txt
%restart_python

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime

In [ ]:
%load_ext autoreload
%autoreload 2

from lakebase_utils import LakebaseConnection

In [ ]:
dbutils.widgets.text("user", "lars.liahagen@databricks.com")
username = dbutils.widgets.get("user")
dbutils.widgets.text("lakebase_instance_name", "smart-stock-db")
lakebase_instance_name = dbutils.widgets.get("lakebase_instance_name")
dbutils.widgets.text("catalog", "smart_stock")
catalog = dbutils.widgets.get("catalog")
dbutils.widgets.text("schema_silver", "smart_stock_silver")
schema_silver = dbutils.widgets.get("schema_silver")
dbutils.widgets.text("schema_gold", "smart_stock_gold")
schema_gold = dbutils.widgets.get("schema_gold")


In [0]:
# =============================================
# CONFIGURATION
# =============================================

# PostgreSQL connection
conn = LakebaseConnection(username, lakebase_instance_name)

print("✅ Setup complete")

✅ Setup complete


In [0]:
def update_dim_warehouses():
    """
    Update warehouse dimension table - Run once daily at 6 AM
    Simple full refresh since warehouses table is small
    """
    
    print(f"\n📦 Updating dim_warehouses at {datetime.now()}")
    
    # Read all warehouses from PostgreSQL
    warehouses_df = conn.execute_query("SELECT * FROM warehouses")
    if warehouses_df.empty:
        print("  ℹ️ No warehouses found")
        return 0
    warehouses_df = spark.createDataFrame(warehouses_df)
    
    # Create dimension with business logic
    dim_warehouses = warehouses_df.select(
        col("warehouse_id"),
        col("name").alias("warehouse_name"),
        col("location"),
        col("manager_id"),
        col("timezone"),
        current_timestamp().alias("last_updated")
    )
    
    # Overwrite dimension table (products don't change often)
    dim_warehouses.write \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(f"`{catalog}`.`{schema_silver}`.dim_warehouses")
    
    record_count = dim_warehouses.count()
    print(f"✅ Updated dim_warehouses with {record_count} warehouses")
    
    return record_count


In [0]:
update_dim_warehouses()


📦 Updating dim_warehouses at 2025-09-16 16:04:11.988894
✅ Updated dim_warehouses with 3 warehouses


3